<a href="https://colab.research.google.com/github/kimlind/ztm-pytorch-for-deeplearning/blob/second-lesson/01_PyTorch_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>